In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import scipy as sp
import math as m
import pickle as pk
import resources.RES as res
# import resources.res.store_interdependencies
from ladybug.sunpath import Sunpath as sp
from sklearn.preprocessing import minmax_scale as sk_minmax
import pymorton as pm
import pygmo as pg

In [2]:
# import lattice
env_lat_path = os.path.relpath("../data/macrovoxels.csv")
envelope_lattice = tg.lattice_from_csv(env_lat_path)

# plot dimensions - USER INPUT
plot_area = 6000

# required FSI - USER INPUT
FSI = 3

area_req = FSI * plot_area
voxs_req = int(area_req / (envelope_lattice.unit[0] * envelope_lattice.unit[1]))

# number of variables:
num_var = envelope_lattice.flatten().shape[0]

# actual PV sun obstructing cost array:
# c1_norm = pk.load(open("../data/c1_norm.pk", "rb"))
sol_interd = pk.load(open("../data/SolG1.pk", "rb")) # interdependencies
sol_blocks = pk.load(open("../data/SolU1.pk", "rb")) # context blocks

# actual daylighting obstructing cost array:
# c2_norm = pk.load(open("../data/c2_norm.pk", "rb"))
sky_interd = pk.load(open("../data/SkyG1.pk", "rb")) # interdependencies
sky_blocks = pk.load(open("../data/SkyU1.pk", "rb")) # context blocks

# actual sky view factor obstructing cost array:
svf_norm = pk.load(open("../data/svf_norm.pk", "rb"))
svf_interd = pk.load(open("../data/SvFG1.pk", "rb")) # interdependencies
svf_blocks = pk.load(open("../data/SvFU1.pk", "rb")) # context blocks

dnr = pk.load(open("../data/dnrval.pk", "rb")) # direct normal radiation

nums = (np.random.rand(125) > 0.5).astype(int) # random numbers for testing functions

In [3]:
def solarhits(interdependencies, contextblocks, radiation, x):
    
    # mask for finding active voxels that may block the rays
    blockedrays = x[np.newaxis, :, np.newaxis] * interdependencies # all rays that are blocked by the 'active' voxels

    blocked = np.sum(blockedrays, axis=1, dtype='int') # how many times each ray is blocked by another active voxel for each voxel
    context_blocks = x[:, np.newaxis] * contextblocks # whether a ray is blocked by the environment for each voxel
    total_blocks = context_blocks + blocked

    dnr_reshape = radiation[np.newaxis, :]

    reaches = np.where(total_blocks == 0, 1, 0) # outputs 1 if a ray can reach the voxel, else it outputs 0
    weighted_hits = dnr_reshape * reaches # optional weighting of the rays

    hits = np.sum(reaches, axis=1) # total number of rays that can reach the current configuration
    directradiation = np.sum(weighted_hits, axis=1) # total direct normal radiation on the voxels TODO: do we want to use this?

    possiblehits = (np.count_nonzero(x) * interdependencies.shape[2]) - context_blocks.sum()
    
    score = hits.sum()/possiblehits
    return score #, weightedscore

In [4]:
def skyhits(interdependencies, contextblocks, x):
    
    # mask for finding active voxels that may block the rays
    blockedrays = x[np.newaxis, :, np.newaxis] * interdependencies # all rays that are blocked by the 'active' voxels

    blocked = np.sum(blockedrays, axis=1, dtype='int') # how many times each ray is blocked by another active voxel for each voxel
    context_blocks = x[:, np.newaxis] * contextblocks # whether a ray is blocked by the environment for each voxel
    total_blocks = context_blocks + blocked

    reaches = np.where(total_blocks == 0, 1, 0) # outputs 1 if a ray can reach the voxel, else it outputs 0

    hits = np.sum(reaches, axis=1) # total number of rays that can reach the current configuration

    possiblehits = (np.count_nonzero(x) * interdependencies.shape[2]) - context_blocks.sum()
    
    score = hits.sum()/possiblehits
    return score #, weightedscore

In [5]:
def reshape_and_store_to_lattice(values_list, envelope_lattice):
    env_all_vox_id = envelope_lattice.indices.flatten()
    env_all_vox = envelope_lattice.flatten() # envelope inclusion condition: True-False
    env_in_vox_id = env_all_vox_id[env_all_vox] # keep in-envelope voxels (True)

    # initialize array
    values_array = np.full(env_all_vox.shape, 0.0)
    
    # store values for the in-envelope voxels
    values_array[env_in_vox_id] = values_list

    # reshape to lattice shape
    values_array_3d = values_array.reshape(envelope_lattice.shape)

    # convert to lattice
    values_lattice = tg.to_lattice(values_array_3d, envelope_lattice)

    return values_lattice

In [6]:
def compactness(x, reference_lattice):
    # create the current configuration as a lattice
    curr_envelope = reshape_and_store_to_lattice(x.astype('bool'), reference_lattice)
    # flatten the envelope
    envlp_voxs = curr_envelope.flatten()

    # create stencil
    stencil = tg.create_stencil("von_neumann", 1, 1)
    stencil.set_index([0,0,0], 0)

    # find indices of the neighbours for each voxel 
    neighs = curr_envelope.find_neighbours(stencil)

    # occupation status for the neighbours for each voxel
    neighs_status = envlp_voxs[neighs]

    # for voxels inside the envelope:
    neigh_array = np.array(neighs_status[envlp_voxs.astype("bool")])  

    # when the neighbour's status is False that refers to an outer face
    outer_faces = np.count_nonzero(neigh_array==0)

    # voxel edge length
    l = envelope_lattice.unit[0] # TODO: can we leave this dimension out?

    # calculate total surface area of outer faces
    A_exterior = (l**2)*outer_faces

    # number of in-envelope voxels
    in_voxels = np.count_nonzero(x)

    # calculate total volume inclosed in the envelope
    V = in_voxels * (l**3)

    # edge length of a cube that has the same volume
    l_ref = V**(1/3)

    # calculate ratio
    R_ref = (6*(l_ref**2))/V

    relative_compactness = (A_exterior/V)/R_ref
    return relative_compactness

In [16]:
class test_python:

    # Number of dimensions
    def __init__(self,dim,envelope):
        self.dim = dim

    # Define objectives    
    def fitness(self, x):
        # direct normal radiation on voxel roofs (PV potential)
        # f1 = -solarhits(sol_interd, sol_blocks, dnr, x)

        # daylighting (sky visibility from building)
        #f2 = -skyhits(sky_interd, sky_blocks, x) # daylighting potential of voxels

        # floor space index
        f3 = -(1 - (abs(voxs_req - sum(x)))/voxs_req) # TODO: this constrains the model too much

        # sky view factor from street level around plot
        f4 = sum(svf_norm[np.nonzero(x)])

        # relative compactness
        #f5 = -compactness(x, envelope_lattice)

        return [f3, f4]
    
    # Return number of objectives
    def get_nobj(self):
        return 2

    # Return bounds of decision variables
    def get_bounds(self):
        return (np.full((self.dim,),0.),np.full((self.dim,),1.))

    # return number of integer variables (all variables are integer in this case TODO: transparency vectors for smoother shapes/results)
    def get_nix(self):
        return self.dim

    # Return function name
    def get_name(self):
        return "Test function MAX no.1"

In [17]:
# create User Defined Problem
prob = pg.problem(test_python(dim = num_var, envelope=envelope_lattice))

In [25]:
# create population
pop = pg.population(prob, size=8)

# select algorithm --> ihs nsga2 maco for integers
# maco seems slow, unreliable
# ihs seems fastest but not always yields results that make sense
# nsga2 is most consistent with results, average speed

# TODO: continuous in stead of discrete/integer
algo = pg.algorithm(pg.nsga2(gen=100))

# run optimization
pop = algo.evolve(pop)

# extract results
fits, vectors = pop.get_f(), pop.get_x()

# extract and print non-dominated fronts
ndf, dl, dc, ndr = pg.fast_non_dominated_sorting(fits)

# ax = pg.plot_non_dominated_fronts(pop.get_f()) # plotting the non dominated fronts #TODO: what exactly does this mean in this context

In [26]:
best = pg.sort_population_mo(points = pop.get_f())[0] # the best solutions (by population)

In [27]:
print("The best configuration is: \n", pop.get_x()[best], "\n It's fitness is: ", pop.get_f()[best], "\n This is population #", best)

The best configuration is: 
 [0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0.
 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1.
 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1.] 
 It's fitness is:  [-1.         24.25954198] 
 This is population # 0


In [28]:
np.count_nonzero(pop.get_x()[best]) # TODO: does the FSI fitness requirement constrain the model too much? 
# TODO: should the other fitness functions be normalized (again) so that all objectives can achieve a maximum of -1? --> increases computation time without changing results

80

In [29]:
def reshape_and_store_to_lattice(values_list, envelope_lattice):
    env_all_vox_id = envelope_lattice.indices.flatten()
    env_all_vox = envelope_lattice.flatten() # envelope inclusion condition: True-False
    env_in_vox_id = env_all_vox_id[env_all_vox] # keep in-envelope voxels (True)

    # initialize array
    values_array = np.full(env_all_vox.shape, 0.0)
    
    # store values for the in-envelope voxels
    values_array[env_in_vox_id] = values_list

    # reshape to lattice shape
    values_array_3d = values_array.reshape(envelope_lattice.shape)

    # convert to lattice
    values_lattice = tg.to_lattice(values_array_3d, envelope_lattice)

    return values_lattice

In [30]:
configuration = reshape_and_store_to_lattice(pop.get_x()[best], envelope_lattice)

In [31]:
# visualize configuration 
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
configuration.fast_vis(p)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(204.88887394336027, 84.88887394336027, 144.88887394336027),
 (60.0, -60.0, 0.0),
 (0.0, 0.0, 1.0)]